<a href="https://colab.research.google.com/github/rgtvlt67/Ecora/blob/main/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import rasterio
from rasterio.features import shapes
from scipy.ndimage import label, gaussian_filter, distance_transform_edt, maximum_filter
import numpy as np
import geopandas as gpd
from shapely.geometry import shape
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Input and output paths
chm_raster_path = "/content/drive/My Drive/chm/chm.tif"  # Replace with your file path
output_shapefile_path = "/content/drive/My Drive/chm/watershed_segments.shp"  # Replace with output path

# Load the Canopy Height Model (CHM)
with rasterio.open(chm_raster_path) as src:
    chm = src.read(1)  # Read the first band
    chm_meta = src.meta  # Get metadata for the raster

# Preprocess CHM (e.g., smoothing)
chm[chm < 0] = 0  # Mask invalid values (e.g., negative heights)
chm_smoothed = gaussian_filter(chm, sigma=2)  # Apply Gaussian smoothing to reduce noise

# Generate markers for watershed using local maxima
footprint = np.ones((3, 3))  # Define a neighborhood size for maximum filter
local_maxima = (chm_smoothed == maximum_filter(chm_smoothed, footprint=footprint))  # Detect local maxima
markers, num_features = label(local_maxima)  # Label the local maxima as markers

# Perform watershed segmentation
from skimage.segmentation import watershed
segmentation = watershed(-chm_smoothed, markers, mask=(chm > 0))  # Watershed on the inverted CHM

# Function to export segmentation results to a shapefile
def export_segments_to_shapefile(segmentation, transform, output_path):
    # Convert segmentation to shapes (polygons)
    results = (
        {'properties': {'segment_id': int(value)}, 'geometry': shape(geom)}
        for geom, value in shapes(segmentation, transform=transform) if value > 0
    )

    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(results, crs="EPSG:3005")

    # Save as shapefile
    gdf.to_file(output_path)
    print(f"Shapefile saved to {output_path}")

# Export the segmentation results
export_segments_to_shapefile(segmentation, src.transform, output_shapefile_path)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Shapefile saved to /content/drive/My Drive/chm/watershed_segments.shp


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 55.6 MB/s eta 0:00:00
